# Abstract

The NessStretch is a refinement of [Paul Nasca](http://www.paulnasca.com/)'s excellent [PaulStretch](http://hypermammut.sourceforge.net/paulstretch/) algorithm.  PaulStretch uses a single frame size throughout the entire frequency range.  The NessStretch's layered analysis bands are a better match for human frequency perception, and do a better job of resolving shorter, noisier high-frequency sounds (sibilance, snares, etc.).

Multiresolution analysis is not a new idea, but to my knowledge this is the first time that it's been applied to audio time-stretching, despite an [uncited claim on Wikipedia](https://en.wikipedia.org/wiki/Audio_time_stretching_and_pitch_scaling#SOLA) that some "high-end commercial audio packages" use wavelets for "high-quality time-stretching."

# Documentation

## Algorithm

The NessStretch algorithm is similar to the [PaulStretch algorithm](http://www.paulnasca.com/algorithms-created-by-me#TOC-PaulStretch-extreme-sound-stretching-algorithm).  The main difference is that NessStretch uses different FFT window sizes for different frequency ranges:

|   window size  |   number of bins  |   low frequency (Hz)  |   high frequency (Hz)  |   window duration (ms)  |   hop duration (ms)  |
|----------------|-------------------|-----------------------|------------------------|-------------------------|----------------------|
|   256          |   64              |   12187.50            |   24000                |   5.33                  |   1.33               |
|   512          |   64              |   6093.75             |   12000                |   10.67                 |   2.67               |
|   1024         |   64              |   3046.88             |   6000                 |   21.33                 |   5.33               |
|   2048         |   64              |   1523.44             |   3000                 |   42.67                 |   10.67              |
|   4096         |   64              |   761.72              |   1500                 |   85.33                 |   21.33              |
|   8192         |   64              |   380.86              |   750                  |   170.67                |   42.67              |
|   16384        |   64              |   190.43              |   375                  |   341.33                |   85.33              |
|   32768        |   64              |   95.21               |   187.5                |   682.67                |   170.67             |
|   65536        |   129             |   0.00                |   93.75                |   1365.33               |   341.33             |

(table caption: default NessStretch analysis settings, assuming a 48000 Hz input file sample rate)

There are 641 frequency bins total, spaced approximately logarithmically through the audible frequency range.  (See the appendix for a full list of analysis bin frequencies with the corresponding pitches.)  This is a pretty good match for the psychometric model of sound perception: "The total number of perceptible pitch steps in the range of human hearing is about 1400" ([Wikipedia](https://en.wikipedia.org/wiki/Just-noticeable_difference#Music_production_applications)).  It's also reasonable from the perspective of tuning theory: the bin spacing varies between a frequency ratio of 65/64 (about $12 \cdot \log_2(65/64) \approx 0.268$ semitones) at the bottom of each band, and 128/127 (about $12 \cdot \log_2(128/127) \approx 0.136$ semitones) at the top.  The algorithm's phase scrambling diffuses the pitch, but the bin spacing ensures that this diffusion will never be too exaggerated.

It's easy to add more frequency resolution (for example, by doubling the size of each window), but the default settings provide a good compromise between pitch resolution and time localization.  The lowest windows are large enough to produce some audible time smearing, but nothing too grotesque, and the highest windows are smooth and responsive, even at 1% of a recording's original speed.

## Implementation

The NessStretch implementation is *somewhat* similar to the PaulStretch stereo Python implementation:

* Both implementations generate timestretch frames by stepping through the input sample array more slowly (by the timestretch factor) than the output sample array.  This creates de facto spectral interpolation.
* Both use convolution with unit-magnitude white noise to randomize bin phases.
* Both use an [RFFT](https://numpy.org/doc/stable/reference/generated/numpy.fft.rfft.html) to optimize analysis and synthesis for real-valued input and output.

There are, however, a couple implementation differences that are not purely cosmetic:

* PaulStretch writes synthesis frames to a buffer, and the buffer content is appended to an output audio file.  This is efficient (there's no need for large intermediate files), but the buffer math is a bit of a headache, and there's no simple way to mix different output layers together.  Instead, NessStretch loads a large mix_bus array for each channel, to which it adds the output from each time-stretched frequency band.  Unless PaulStretch, this generates some large intermediate files (roughly 10 MB per channel per minute), but the process is more transparent, and mixing the frequency bands together is trivial.
*  PaulStretch doesn't normalize the output audio (which makes sense, because there's no simple way to normalize an audio file "in real time"; you would have to use some sort of dynamics processing).  NessStretch normalizes the maximum output to the maximum input (all  the audio data is stored in arrays ahead of time, so this is easy to do).

Some miscellaneous script details that may not be obvious:

* RFFT bins: an RFFT returns nfft // 2 + 1 bins total.  Bin 0 is the DC component (0 Hz), and bin nfft // 2 is the Nyquist component (sampling rate / 2 Hz).
* Input file padding: pad the input audio with nfft // 2 samples on either end to center the analysis windows correctly.  (It's easy to check this by time-stretching an impulse signal: the output should sound like a symmetrical filter sweep.)
* fancy_bands: the bin range tuple is designed like a Python range argument.  (The high bin **index** is actually 128.)
* target_length: I'm not sure if this calculation is exactly right, but it works.

# Examples

* [You stay on my mind before I wake up](https://alexness.bandcamp.com/album/you-stay-on-my-mind-before-i-wake-up)


# Appendix: list of analysis frequencies and pitches

In [66]:
import numpy as np

fancy_bands = {
    256: (65, 129),
    512: (65, 129),
    1024: (65, 129),
    2048: (65, 129),
    4096: (65, 129),
    8192: (65, 129),
    16384: (65, 129),
    32768: (65, 129),
    65536: (0, 129)
    }
input_sample_rate = 48000
freqs = []
for window_size in reversed(sorted(fancy_bands.keys())):
    low_bin, high_bin = fancy_bands[window_size]
    band_freqs = np.fft.rfftfreq(window_size, 1/input_sample_rate)[low_bin:high_bin]
    freqs.extend(band_freqs)
    
# adapted from
# https://www.johndcook.com/blog/2016/02/10/musical-pitch-notation/
A4 = 440
C0 = A4*pow(2, -4.75)  
names = ["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B"]

class Pitch(object):
    def __init__(self, freq):
        self.freq = freq
        self.step, self.microtone = divmod(12 * np.log2(freq/C0), 1)
        self.cents_deviation = 100 * self.microtone
        self.octave, n = [int(i) for i in divmod(self.step, 12)]
        self.name = names[int(n)]
    def __str__(self):
        return f'{self.freq:10.2f} {self.name:2} {self.cents_deviation:5.2f} {self.octave:3}'

for i, f in enumerate(freqs[1:]):
    p = Pitch(f)
    print(f'{i+1:4}{p}')

   1      0.73 F# 23.26  -5
   2      1.46 F# 23.26  -4
   3      2.20 C# 25.22  -3
   4      2.93 F# 23.26  -3
   5      3.66 A#  9.58  -3
   6      4.39 C# 25.22  -2
   7      5.13 D# 92.09  -2
   8      5.86 F# 23.26  -2
   9      6.59 G# 27.17  -2
  10      7.32 A#  9.58  -2
  11      8.06 B  74.58  -2
  12      8.79 C# 25.22  -1
  13      9.52 D  63.79  -1
  14     10.25 D# 92.09  -1
  15     10.99 F  11.53  -1
  16     11.72 F# 23.26  -1
  17     12.45 G  28.22  -1
  18     13.18 G# 27.17  -1
  19     13.92 A  20.78  -1
  20     14.65 A#  9.58  -1
  21     15.38 A# 94.05  -1
  22     16.11 B  74.58  -1
  23     16.85 C  51.54   0
  24     17.58 C# 25.22   0
  25     18.31 C# 95.89   0
  26     19.04 D  63.79   0
  27     19.78 D# 29.13   0
  28     20.51 D# 92.09   0
  29     21.24 E  52.84   0
  30     21.97 F  11.53   0
  31     22.71 F  68.30   0
  32     23.44 F# 23.26   0
  33     24.17 F# 76.54   0
  34     24.90 G  28.22   0
  35     25.63 G  78.40   0
  36     26.37 G# 27